# RNN Basic
 
1) BasicRNNCell  
2) dynamic_rnn

In [1]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
###Disable Warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import logging
tf.get_logger().setLevel(logging.ERROR)
#import sys
#original_stdout = sys.stdout
#sys.stdout = None
#import warnings
#warnings.filterwarnings("ignore")
import numpy as np
from tensorflow.contrib import rnn
import pprint #Data pretty printer
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [2]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

#4종류이므로, input dimension: 4

![image](https://cloud.githubusercontent.com/assets/901975/23348727/cc981856-fce7-11e6-83ea-4b187473466b.png)

In [3]:
#입출력 테스트: 학습과정 아님
with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2 # cell의 출력(output) 결과값이 몇개의 element인가를 결정: (?,?,2)
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size) 
            #SimpleRNN 대신 다른 Cell로도 간편하게 변경 가능. e.g. BasicLSTMCell()
    print(cell.output_size, cell.state_size) 
        #output size =2(hidden_size에 의해 결정) / state size는 무엇?

    x_data = np.array([[h]], dtype=np.float32) 
    pp.pprint(x_data) # x_data = [[[1,0,0,0]]]
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
'''
2 2
array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[ 0.28406101,  0.53163123]]], dtype=float32)
'''
pass

2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
array([[[-0.4800782 ,  0.07582483]]], dtype=float32)


---
여러개의 값 받기(Sequence data)  
sequence_length는 이것의 길이가 몇개인가를 나타내며, shape의 두번째 인자가 sequence_length임.  
이것을 정의해주면 알아서 sequence가 펼쳐진다.  
?: 아래에 x0, x1, x0, x1, x1인 이유는 언급이 없으니 pass. 큰 의미는 없어보임  


![image](https://cloud.githubusercontent.com/assets/901975/23383634/649efd0a-fd82-11e6-925d-8041242743b0.png)

In [ ]:
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data) #입력 출력
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval()) #출력이 어떻게 나오는지 확인
    '''
    (1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
array([[[ 0.50641137,  0.55783308],
        [-0.61545879, -0.04334207],
        [-0.77109283, -0.23411733],
        [-0.76478487, -0.07172935],
        [-0.72683465,  0.5266667 ]]], dtype=float32)
    '''

---
문자열을 여러개 입력해주는 방법: batch_size (아래 그림에서는 batch)  
빠르게 여러 문자열을 셀에 입력해줄 수 있다. 가장 첫번째 shape가 batch_size  

![image](https://cloud.githubusercontent.com/assets/901975/23383681/9943a9fc-fd82-11e6-8121-bd187994e249.png)

In [4]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32) 
                       #batch=3(row), sequence=5(col), dim=4(h, e, l, o) => (3,5,4)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

'''
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ 0.05147979, -0.12499733],
        [-0.03096316,  0.0606184 ],
        [ 0.07608285, -0.03678102],
        [ 0.13522112, -0.0920362 ],
        [ 0.0235186 ,  0.02936662]],

       [[-0.08701393,  0.11020429],
        [-0.10396396,  0.17681563],
        [ 0.04137699,  0.01251686],
        [ 0.11497379, -0.06123064],
        [ 0.15778907, -0.10736831]],

       [[ 0.09156778, -0.06834684],
        [ 0.14449464, -0.11151826],
        [ 0.0351325 ,  0.04766054],
        [-0.06127799,  0.13465708],
        [ 0.06125095,  0.00202556]]], dtype=float32)
'''
pass

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
array([[[ 0.12912811,  0.16881344],
        [ 0.15748908,  0.07690624],
        [ 0.09947554, -0.00030735],
        [-0.01995066, -0.07370917],
        [-0.11888505,  0.07191802]],

       [[ 0.12281438, -0.00911933],
        [-0.01282248,  0.11662932],
        [-0.100